In [ ]:
# Step 1: Import Libraries
from arango import ArangoClient
import networkx as nx

In [ ]:
# Step 2: Define the Function
def persist_graph_to_arangodb(graph, db_name="graph_db", graph_name="ai_incidents_graph"):
    """
    Persist a NetworkX graph to ArangoDB.
    """
    try:
        # Initialize the ArangoDB client
        client = ArangoClient(hosts="http://localhost:8529")

        # Connect to the system database
        sys_db = client.db("_system", username="root", password="passwd")

        # Create the database if it doesn't exist
        if not sys_db.has_database(db_name):
            sys_db.create_database(db_name)
            print(f"Database '{db_name}' created.")

        # Connect to the target database
        db = client.db(db_name, username="root", password="passwd")

        # Create the graph if it doesn't exist
        if not db.has_graph(graph_name):
            graph_collection = db.create_graph(graph_name)
            print(f"Graph '{graph_name}' created.")

        # Get the graph and collections
        graph_collection = db.graph(graph_name)
        if not graph_collection.has_vertex_collection("nodes"):
            graph_collection.create_vertex_collection("nodes")
            print("Collection 'nodes' created.")

        if not graph_collection.has_edge_collection("edges"):
            graph_collection.create_edge_definition(
                edge_collection="edges",
                from_vertex_collections=["nodes"],
                to_vertex_collections=["nodes"]
            )
            print("Collection 'edges' created.")

        # Add nodes to ArangoDB
        nodes_collection = db.collection("nodes")
        for node in graph.nodes():
            nodes_collection.insert({"_key": str(node)})

        # Add edges to ArangoDB
        edges_collection = db.collection("edges")
        for edge in graph.edges(data=True):
            edges_collection.insert({
                "_from": f"nodes/{edge[0]}",
                "_to": f"nodes/{edge[1]}",
                "weight": edge[2].get("weight", 1)
            })

        print(f"Graph '{graph_name}' loaded into ArangoDB.")

    except Exception as e:
        print(f"Error loading graph into ArangoDB: {e}")

In [ ]:
# Step 3: Example Usage
# Load the graph from the saved file
G = nx.read_graphml("graph.graphml")

# Persist the graph to ArangoDB
persist_graph_to_arangodb(G)

NameError: name 'nx' is not defined